# Forward auto-diff through Dual Numbers

In [2]:
import numpy as np

Let us define a class `DualNumber` that represents a dual number

$$
a + b \epsilon 
$$

where $a$ is the "real" part and $b$ is the "dual" part.

In [3]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

Implement the operator sum `__add__` for this class.

In [4]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

  def __add__(self, other):
    # implement the operation "self + other"
    return DualNumber(self.real + other.real, self.dual + other.dual)

Define the dual numbers

$$
\begin{split}
x &= 1 + 2 \epsilon \\
y &= 1.5 + 3.1 \epsilon \\
\end{split}
$$

Then, compute $z = x + y$ and display the result.

In [5]:
x = DualNumber(1, 2)
y = DualNumber(1.5, 3.1)

z = x + y
z

2.5 + 5.1 epsilon

Define now the dual number

$$
\begin{split}
x &= 1 + 2 \epsilon \\
\end{split}
$$

and try to compute $w = x + 1$. What is going on?

In [6]:
x = DualNumber(1, 2)

w = x + 1
w

AttributeError: 'int' object has no attribute 'dual'

Try now to compute $w = 1 + x$ (in this specific order). What is going on this time?

In [7]:
w = 1 + x
w

TypeError: unsupported operand type(s) for +: 'int' and 'DualNumber'

To overcome the above inconvenient, introduce a check (inside the definition of `__add__`) on the type of `other`. Moroever, define the operator `__radd__`, besides `__add__`.

In [8]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'

  def __add__(self, other):
    # implement the operation "self + other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real + other.real, self.dual + other.dual)
    elif isinstance(other, (int, float)):
      return DualNumber(self.real + other, self.dual)
    else:
      throw(TypeError('unsupported operand type(s) for +: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __radd__(self, other):
    # implement the operation "other + self"
    return self.__add__(other)

Try again to compute $w = x + 1$

In [9]:
x = DualNumber(1, 2)
w = x + 1
w

2 + 2 epsilon

Try again to compute $w = 1 + x$

In [10]:
w = 1 + x
w

2 + 2 epsilon

Now that we have learnt how to treat the operator "+", let us define the full class `DualNumber`, implementing also the operators "-", "*", "/", "**".

In [11]:
class DualNumber:
  def __init__(self, real, dual):
    # dual number: 'real' + 'dual' * eps
    self.real = real
    self.dual = dual

  def __add__(self, other):
    # implement the operation "self + other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real + other.real, self.dual + other.dual)
    elif isinstance(other, (int, float)):
      return DualNumber(self.real + other, self.dual)
    else:
      throw(TypeError('unsupported operand type(s) for +: \'DualNumber\' and \'' + type(other).__name__ + '\''))
  
  def __radd__(self, other):
    # implement the operation "other + self"
    return self.__add__(other)

  def __sub__(self, other):
    # implement the operation "self + other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real - other.real, self.dual - other.dual)
    elif isinstance(other, (int, float)):
      return DualNumber(self.real - other, self.dual)
    else:
      throw(TypeError('unsupported operand type(s) for -: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __rsub__(self, other):
    if isinstance(other, DualNumber):
      return other - self
    elif isinstance(other, (int, float)):
      return DualNumber(other, 0.0) - self
    else:
      throw(TypeError('unsupported operand type(s) for -: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __mul__(self, other):
    # implement the operation "self * other"
    if isinstance(other, DualNumber):
      return DualNumber(self.real * other.real, self.real * other.dual + self.dual * other.real)
    elif isinstance(other, (int, float)):
      return DualNumber(self.real * other, self.dual * other)
    else:
      throw(TypeError('unsupported operand type(s) for *: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __rmul__(self, other):
    # implement the operation "other * self"
    return self.__mul__(other)

  def __truediv__(self, other):
    # implement the operation "self / other"
    # (a +be) / (c + de) = (a + be)(c-de) / (c + de)(c - de) = (ac - ade + bce - 0) / (c^2 - 0) =  ac/c^2 + (bce - ade)/c^2 = a/c + (bc - ad)/c^2 e
    if isinstance(other, DualNumber):
      return DualNumber(self.real / other.real, (self.dual * other.real - self.real * other.dual) / (other.real ** 2))
    elif isinstance(other, (int, float)):
      return (1/other) * self
    else:
      throw(TypeError('unsupported operand type(s) for /: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __rtruediv__(self, other):
    if (isinstance(other, DualNumber)):
      return other / self
    elif isinstance(other, (int, float)):
      return DualNumber(other, 0.0).__truediv__(self)
    else:
      throw(TypeError('unsupported operand type(s) for /: \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __pow__(self, other):
    # implement the operation "self ** other"
    # (a + be)(a + be) = a^2 + 2abe + b^2e^2 = a^2 + 2abe
    # (a^2 + 2abe)(a + be) = a^3 + a^2be + 2a^2be + 2ab^2e^2 = a^3 + 3a^2be
    # (a^3 + 3a^2be)(a + be) = a^4 + a^3be + 3a^3be + 3a^2b^2e^2 = a^4 + 4a^3be
    # ... (a + be) ^ n = a^n + n * a^(n-1) * be
    if isinstance(other, (int, float)):
      return DualNumber(self.real ** other, other * self.real ** (other - 1) * self.dual)
    else:
      throw(TypeError('unsupported operand type(s) for ** or pow(): \'DualNumber\' and \'' + type(other).__name__ + '\''))

  def __repr__(self):
    return repr(self.real) + ' + ' + repr(self.dual) + ' epsilon'


Define the dual numbers

$$
\begin{split}
x &= 1 + 2 \epsilon \\
y &= 1.5 + 3.1 \epsilon \\
\end{split}
$$

Then, compute the result of the following operations:
- $x + y$
- $x - y$
- $x y$
- $x / y$
- $x + 1$
- $2 x$
- $x ^ 3$

In [12]:
x = DualNumber(1, 2)
y = DualNumber(1.5, 3.1)

In [13]:
print(f"""The results of the operations are:
      ({x}) + ({y}) = {x + y}\n
      ({x}) - ({y}) = {x - y}\n
      ({x}) * ({y}) = {x * y}\n
      ({x}) / ({y}) = {x / y}\n
      ({x}) + 1 = {x + 1}\n
      2({x}) = {2 * x}\n
      ({x})**3 = {x ** 3}\n
      """)

The results of the operations are:
      (1 + 2 epsilon) + (1.5 + 3.1 epsilon) = 2.5 + 5.1 epsilon

      (1 + 2 epsilon) - (1.5 + 3.1 epsilon) = -0.5 + -1.1 epsilon

      (1 + 2 epsilon) * (1.5 + 3.1 epsilon) = 1.5 + 6.1 epsilon

      (1 + 2 epsilon) / (1.5 + 3.1 epsilon) = 0.6666666666666666 + -0.04444444444444448 epsilon

      (1 + 2 epsilon) + 1 = 2 + 2 epsilon

      2(1 + 2 epsilon) = 2 + 4 epsilon

      (1 + 2 epsilon)**3 = 1 + 6 epsilon

      


Define now the functions `my_sin`, `my_cos` and `my_exp`, implementing the operations sinus, cosinus and exponential, respectively.

In [14]:
def my_sin(x):
  if isinstance(x, DualNumber):
    return DualNumber(np.sin(x.real), np.cos(x.real) * x.dual)
  else:
    return np.sin(x)

# x = a + be
# sin x = x - x^3/3! + x^5/5! - x^7/7! + ...
# cos x = 1 - x^2/2! + x^4/4! - x^6/6! + ...
#       = 1 - (a+be)^2/2! + (a+be)^4/4! - (a+be)^6/6! + ...
#       = 1 - (a^2 + 2abe)/2! + (a^4 + 4a^3be)/4! - (a^6 + 6a^5be)/6! + ...
#       = (1 - a^2/2! + a^4/4! - a^6/6! + ...) - be(2a/2! - 4a^3/4! + 6a^5/6! - ...)
#       = (1 - a^2/2! + a^4/4! - a^6/6! + ...) - be(a/1! - a^3/3! + a^5/5! - ...)
#       = cos a - (bsin a)e

def my_cos(x):
  if isinstance(x, DualNumber):
    return DualNumber(np.cos(x.real), -np.sin(x.real) * x.dual)
  else:
    return np.cos(x)

# x = a + be
# exp x = 1 + x + x^2/2! + x^3/3! + ...
#       = 1 + (a+be) + (a+be)^2/2! + (a+be)^3/3! + ...
#       = 1 + a + be + (a^2 + 2abe)/2! + (a^3 + 3a^2be)/3! + ...
#       = (1 + a + a^2/2! + a^3/3! + ...) + (b + ab + a^2b/2! + a^3b/3! + ...)
#       = exp a + be(1 + a/1! + a^2/2! + a^3/3! + ...)
#       = exp a + be exp a

def my_exp(x):
  if isinstance(x, DualNumber):
    return DualNumber(np.exp(x.real), np.exp(x.real) * x.dual)
  else:
    return np.exp(x)

Define the dual number

$$
\begin{split}
x &= 1 + 2.3 \epsilon \\
\end{split}
$$

Then, compute the result of the following operations:
- $\sin(x)$
- $\exp(x)$

In [15]:
x = DualNumber(1, 2.3)
print(f"""The results of the operations are:
      sin({x}) = {my_sin(x)}\n
      cos({x}) = {my_cos(x)}\n
      exp({x}) = {my_exp(x)}\n
      """)

The results of the operations are:
      sin(1 + 2.3 epsilon) = 0.8414709848078965 + 1.2426953034967214 epsilon

      cos(1 + 2.3 epsilon) = 0.5403023058681398 + -1.9353832650581617 epsilon

      exp(1 + 2.3 epsilon) = 2.718281828459045 + 6.252048205455803 epsilon

      


Define now a function `auto_diff` that, given a function $f \colon \mathbb{R} \to \mathbb{R}$ and a real number $x$, returns $f'(x)$, exploiting the class `DualNumber`. The function must have the following signature:
```python
def auto_diff(f, x):
  ...
```


In [16]:
# auto_diff function returns the derivative of the function f at x
def auto_diff(f, x):
  return f(DualNumber(x, 1)).dual

Consider the function 

$$
f(x) = x \sin(x^2)
$$

and use the function implemented above to compute $f'(x_0)$ for $x_0 = 0.13$. Compare the result with the analytical solution and compute the relative error.



In [35]:
func = lambda x : x * my_sin(x ** 2)
x0 = 0.13
df_ad = auto_diff(func, x0)
print(f"""The derivative of the function at {x0} with AD is {df_ad}""")

func_diff = lambda x : np.sin(x ** 2) + 2 * x ** 2 * np.cos(x ** 2)
df_diff = func_diff(x0)
print(f"""The exact derivative of the function at {x0} is {df_diff}""")

#error
print(f"""The error is {abs(df_ad - df_diff)}""")

The derivative of the function at 0.13 with AD is 0.050694368849202455
The exact derivative of the function at 0.13 is 0.05069436884920245
The error is 6.938893903907228e-18


Repeat the previous point, this time by computing the numerical derivative (i.e. through finite differences).

In [36]:
import scipy.misc
df_FD = scipy.misc.derivative(func, x0, dx=1e-6)
df_FD

print(f"""The derivative of the function at {x0} with FD is {df_FD}""")
print(f"""The error is {abs(df_FD - df_diff)}""")

The derivative of the function at 0.13 with FD is 0.05069436885031531
The error is 1.1128598043086413e-12


C:\Users\nrtc\AppData\Local\Temp\ipykernel_15556\4083785684.py:2: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  df_FD = scipy.misc.derivative(func, x0, dx=1e-6)


Repeat the previous point, this time by computing the symbolic derivative (module `sympy` = **sym**bolic **py**thon)

In [37]:
import sympy
x = sympy.symbols('x')
func_sym = x*sympy.sin(x**2)
x0 = 0.13

dfunc_sym = sympy.diff(func_sym, x)
print(dfunc_sym)
df_sy = dfunc_sym.subs(x, x0)
df_sy

print(f"""The derivative of the function at {x0} with symbolic differentiation is {df_sy}""")
print(f"""The error is {abs(df_sy - df_diff)}""")

2*x**2*cos(x**2) + sin(x**2)
The derivative of the function at 0.13 with symbolic differentiation is 0.0506943688492024
The error is 0


Evaluate and compare the execution time of the different approaches.
To compute the execution time of a line of code, prepend IPython [magic command](https://ipython.readthedocs.io/en/stable/interactive/magics.html) `%timeit` to the line.

Example:
```python
%timeit np.random.rand(1000)
```

In [24]:
%timeit auto_diff(func, x0)

3.27 µs ± 34.1 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [25]:
%timeit df_FD = scipy.misc.derivative(func, x0, dx=1e-6)

<magic-timeit>:1: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools


17.1 µs ± 1.19 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [27]:
%timeit dfunc_sym = sympy.diff(func_sym, x)

67.1 µs ± 885 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [26]:
%timeit df_sy = dfunc_sym.subs(x, x0)

24.4 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


Consider now the function 
$$
f(x) = \frac{1}{x^5}
$$
compute the derivative in the point $x_0 = 10^{-2}$ with AD and FD and compare the results with the exact solution.

In [29]:
f = lambda x : 1 / (x ** 5)
x0 = 10e-2

df_ad = auto_diff(f, x0)
df_fd = scipy.misc.derivative(f, x0, dx=1e-6)
df_sy = sympy.diff(f(x), x).subs(x, x0)

print(f"""The results of the operations are:
        f\'({x0}) (AD): {df_ad}\n
        f\'({x0}) (FD): {df_fd}\n
        f\'({x0}) (EXACT): {df_sy}\n
        """)
# the errors of AD and FD w.r.t the exact derivative
print(f"""The errors of AD and FD w.r.t the exact derivative are:
        err (AD): {abs(df_ad - df_sy)/abs(df_sy)}\n
        err (FD): {abs(df_fd - df_sy)/abs(df_sy)}\n
        """)

The results of the operations are:
        f'(0.1) (AD): -4999999.999999998

        f'(0.1) (FD): -5000000.003499736

        f'(0.1) (EXACT): -5000000.00000000

        
The errors of AD and FD w.r.t the exact derivative are:
        err (AD): 0

        err (FD): 6.99947588145733E-10

        


C:\Users\nrtc\AppData\Local\Temp\ipykernel_15556\1605277407.py:5: DeprecationWarning: scipy.misc.derivative is deprecated in SciPy v1.10.0; and will be completely removed in SciPy v1.12.0. You may consider using findiff: https://github.com/maroba/findiff or numdifftools: https://github.com/pbrod/numdifftools
  df_fd = scipy.misc.derivative(f, x0, dx=1e-6)
